In [1]:
import pandas as pd

In [2]:
CtFamPens = 0.8
NroBenAno = 13
FtBenEnti = 0.98
FtBenLiquido = 1
peculioMorteAssistido = 11.62
perct_saque_bua = 0
perct_opcao_bua = 0
CdPlanBen = 5
teto_contribuicao_inss = 5481.59
faixa_1 = 0
faixa_2 = 0
faixa_3 = 0

In [12]:
def calcular_beneficio_total_atc(row):
    benef_total_atc = 0
    
    if CdPlanBen == 1:
        benef_total_atc = max(0, row['SalConPrjEvol'] - row['SalProjeInssEvol'])

        if row['PeFatReduPbe'] > 0:
            benef_total_atc = max(0, benef_total_atc * row['PeFatReduPbe'])

        FtRenVitAtc = max(0, (row['axcb'] + CtFamPens * row['PrbCasado'] * (row['ajxcb'] - row['ajxx'])) * NroBenAno * FtBenEnti)

        if FtRenVitAtc > 0:
            benef_total_atc = max(benef_total_atc, ((row['VlSdoConPartEvol'] + row['VlSdoConPatrEvol']) / FtRenVitAtc) * FtBenEnti)
    elif CdPlanBen == 2:
        benef_total_atc = max(0, row['VlBenSaldado'] * FtBenLiquido * FtBenEnti)
    elif CdPlanBen == 4 or CdPlanBen == 5:
        FtRenVitAtc = max(0, (row['axcb'] + CtFamPens * row['PrbCasado'] * (row['ajxcb'] - row['ajxx'])) * NroBenAno * FtBenEnti + (row['Ax'] * peculioMorteAssistido))

        if FtRenVitAtc > 0:
            benef_total_atc = max(0, ((row['VlSdoConPartEvol'] + row['VlSdoConPatrEvol']) / FtRenVitAtc) * FtBenEnti)
            
    return round(benef_total_atc, 2)

In [13]:
def calcular_contribuicao(beneficio, teto_contribuicao_inss, faixa_1, faixa_2, faixa_3):
    contribuicao = 0

    if CdPlanBen == 1:
        contr1, contr2, contr3 = 0

        contr1 = min(beneficio, teto_contribuicao_inss / 2) * faixa_1

        if (beneficio > (teto_contribuicao_inss / 2)):
            contr2 = (min(beneficio, teto_contribuicao_inss) - (teto_contribuicao_inss / 2)) * faixa_2

        if (beneficio > teto_contribuicao_inss):
            contr3 = (beneficio - teto_contribuicao_inss) * faixa_3

        contribuicao = contr1 + contr2 + contr3

    return round(contribuicao, 2)

In [14]:
def calcular_beneficio_liquido_atc(beneficio_total, contribuicao, perct_opcao_bua, perct_saque_bua):
    benef_liquido_atc = max(0, (beneficio_total - contribuicao) * (1 - perct_opcao_bua * perct_saque_bua))
    return round(benef_liquido_atc, 2)

In [3]:
ativo = pd.read_csv("input_csv/30318/atc_ativos.csv", sep=",", decimal=".").drop(['id_bloco'], axis=1)

In [4]:
ativo['SalConPrjEvol'] = [float(x.replace(".","").replace(",", ".")) for x in ativo['SalConPrjEvol']]

In [5]:
ativo['SalProjeInssEvol'] = [float(x.replace(".","").replace(",", ".")) for x in ativo['SalProjeInssEvol']]

In [6]:
ativo['VlSdoConPartEvol'] = [float(x.replace(".","").replace(",", ".")) for x in ativo['VlSdoConPartEvol']]

In [7]:
ativo['VlSdoConPatrEvol'] = [float(x.replace(".","").replace(",", ".")) for x in ativo['VlSdoConPatrEvol']]

In [8]:
ativo['VlBenSaldado'] = [float(x.replace(".","").replace(",", ".")) for x in ativo['VlBenSaldado']]

In [9]:
ativo.dtypes

ID_PARTICIPANTE       int64
t                     int64
SalConPrjEvol       float64
SalProjeInssEvol    float64
VlSdoConPartEvol    float64
VlSdoConPatrEvol    float64
VlBenSaldado        float64
PeFatReduPbe        float64
PrbCasado           float64
axcb                float64
ajxcb               float64
ajxx                float64
dy_dx               float64
Ax                  float64
apx                 float64
dtype: object

In [10]:
ativo.describe()

,ID_PARTICIPANTE,t,SalConPrjEvol,SalProjeInssEvol,VlSdoConPartEvol,VlSdoConPatrEvol,VlBenSaldado,PeFatReduPbe,PrbCasado,axcb,ajxcb,ajxx,dy_dx,Ax,apx
count,6.999713e+06,6.999713e+06,6.999713e+06,6.999713e+06,6.999713e+06,6.999713e+06,6.999713e+06,6.999713e+06,6.999713e+06,6.999713e+06,6.999713e+06,6.999713e+06,6.999713e+06,6.999713e+06,6.999713e+06
mean,1.345349e+07,4.226961e+01,2.605807e+04,2.718294e+03,1.949767e+04,1.919802e+04,3.796256e+02,3.741468e-04,7.362142e-01,7.480054e+00,7.899173e+00,6.207645e+00,1.190426e-01,5.391682e-01,1.185091e-02
std,2.344467e+04,2.530643e+01,2.403347e+04,8.791827e+02,7.543883e+04,7.449484e+04,8.743477e+02,1.702829e-02,1.244960e-01,5.335073e+00,5.387815e+00,5.206425e+00,2.401038e-01,2.906836e-01,1.082149e-01
min,1.341059e+07,0.000000e+00,0.000000e+00,9.285800e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.343374e+07,2.100000e+01,9.932580e+03,2.250910e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e-01,2.529665e+00,2.750285e+00,1.580103e+00,1.607000e-05,2.747919e-01,0.000000e+00
50%,1.345418e+07,4.200000e+01,1.884815e+04,2.728950e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.500000e-01,6.033080e+00,6.839500e+00,4.245058e+00,2.045090e-03,5.954860e-01,0.000000e+00
75%,1.347376e+07,6.300000e+01,3.413308e+04,3.212460e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.500000e-01,1.242805e+01,1.294330e+01,1.066725e+01,7.305377e-02,8.244452e-01,0.000000e+00
max,1.349354e+07,1.050000e+02,3.272217e+05,5.481590e+03,2.193858e+06,1.938484e+06,1.303134e+04,9.760000e-01,8.500000e-01,1.793904e+01,1.807460e+01,1.756892e+01,1.000000e+00,9.477775e-01,1.000000e+00


In [11]:
ativo.head()

,ID_PARTICIPANTE,t,SalConPrjEvol,SalProjeInssEvol,VlSdoConPartEvol,VlSdoConPatrEvol,VlBenSaldado,PeFatReduPbe,PrbCasado,axcb,ajxcb,ajxx,dy_dx,Ax,apx
0,13410587,0,21844.30,2953.53,156073.53,150133.2,4192.86,0.0,0.6,14.273423,13.189854,12.124317,1.000000,0.230670,1.0
1,13410587,1,22366.38,2953.53,0.00,0.0,4192.86,0.0,0.6,14.065947,12.943303,11.844705,0.837931,0.241505,0.0
2,13410587,2,22900.94,2953.53,0.00,0.0,4192.86,0.0,0.6,13.850949,12.689163,11.557927,0.671063,0.252733,0.0
3,13410587,3,23448.27,2953.53,0.00,0.0,4192.86,0.0,0.6,13.628501,12.428045,11.264793,0.534313,0.264350,0.0
4,13410587,4,24008.68,2953.53,0.00,0.0,4192.86,0.0,0.6,13.398817,12.160474,10.966132,0.436876,0.276344,0.0


In [15]:
ativo['beneficio_total_atc'] = ativo.apply(calcular_beneficio_total_atc, axis = 1)

In [16]:
ativo['contribuicao_beneficio_atc'] = [calcular_contribuicao(value, teto_contribuicao_inss, faixa_1, faixa_2, faixa_3) for value in ativo['beneficio_total_atc']]

In [17]:
ativo['beneficio_liquido_atc'] = [calcular_beneficio_liquido_atc(b, c, perct_opcao_bua, perct_saque_bua) for b, c in zip(ativo['beneficio_total_atc'], ativo['contribuicao_beneficio_atc'])]

In [19]:
 ativo[['ID_PARTICIPANTE', 't', 'beneficio_total_atc', 'contribuicao_beneficio_atc', 'beneficio_liquido_atc']]

,ID_PARTICIPANTE,t,beneficio_total_atc,contribuicao_beneficio_atc,beneficio_liquido_atc
0,13410587,0,1570.79,0,1570.79
1,13410587,1,0.00,0,0.00
2,13410587,2,0.00,0,0.00
3,13410587,3,0.00,0,0.00
4,13410587,4,0.00,0,0.00
5,13410587,5,0.00,0,0.00
6,13410587,6,0.00,0,0.00
7,13410587,7,0.00,0,0.00
8,13410587,8,0.00,0,0.00
9,13410587,9,0.00,0,0.00
